# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [2]:
import pandas as pd
df = pd.read_csv('/datasets/data.csv')
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Прошу прощения за опечатки. Что-то не могу настроить проверку правописания здесь, хотя на других страницах проверяет. ***

In [3]:
df.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Проблема: 
пропущены значения в двух столбцах: days_employed общий трудовой стаж и total_income ежемесячный доход. 

Причины:  
по обоим параметрам, скорее всего не предоставлены справки о доходах с места работы, доход не подтвержден. А параметр income_type, тип занятости, записан со слов клиентов без подтверждения, поэтому в нем пропуском нет.

Подход к решению проблемы: 
1) Общий трудовой стаж у нас не фигурирует в качестве переменной в перечне вопросов от которой зависит успешность погашения кредита.
2) Доход является важнейшей частью кредитного скоринга. Можно записать нулевой доход, но это неправильно, так как люди на что-то живут и какой-то доход есть, но они, наверное, его не могут подтвердить. Записать средний тоже неправильно, так как какие на это есть основания? Если это теневая занятость, то она характеризуется более нестабильным доходом. С другой стороны банк работает с клиентами без подтверждения дохода, очевидно, такие клиенты являются более рискованными. Но банк, желая расширить количество потенциальных клиентов, вынужден работать и с такими клиента, увеличивая свои риски за счет возможного повышения доходности. 

Предлагаемое решение: 
1. Для общего трудового стажа пропуски заменим на 0. Так как в колонке со стажем данные некорректы. 
2. Для ежемесячного дохода заменин пропуски на нули, имея в виду, что наши статистические характеристики будут смещены из-за нулевых доходов. Поэтому с нулевыми доходами надо работать отдельно, понимая что это особая категория, потенциально более рискованных клиентов.   


In [4]:
df_nan = df[df['days_employed'].isna()]
df_nan = df[df['total_income'].isna()]
df_nan.count()

children            2174
days_employed          0
dob_years           2174
education           2174
education_id        2174
family_status       2174
family_status_id    2174
gender              2174
income_type         2174
debt                2174
total_income           0
purpose             2174
dtype: int64

Там где пропущен days_employed также пропущен и total_income ежемесячный доход и наоборот. Предположение о причинах пропусков в виде не предоставления документов с места работы становиться в этом случае еще более вероятным.


Из-за наличия NaN, пропущенных значений столбцы days_employed и total_income имеют тип float64. Если для последнего это вполне может быть, то для первого это некорректно. 


In [5]:
df['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

В столбце children выявлены отрицательные значения и крайне маловероятное количество детей 20 при отсутствии строк c количеством детей в диапазоне от 6 до 19. Поэтому наиболее вероятно, что -1 это 1, а 20 это 2. 

In [6]:
df['days_employed'] = df['days_employed'].apply(abs) #уберем отрицательные значения
df['days_employed'].describe()

count     19351.000000
mean      66914.728907
std      139030.880527
min          24.141633
25%         927.009265
50%        2194.220567
75%        5537.882441
max      401755.400475
Name: days_employed, dtype: float64

In [7]:
df['days_employed'].abs()

0          8437.673028
1          4024.803754
2          5623.422610
3          4124.747207
4        340266.072047
             ...      
21520      4529.316663
21521    343937.404131
21522      2113.346888
21523      3112.481705
21524      1984.507589
Name: days_employed, Length: 21525, dtype: float64

Медианный трудовой стаж составляет 183 года. Должно быть у нас клиенты эльфу и для них более высокий срок выхода на пенсию))
Столбец с трудовым стражем заполнен некорректо, видимо, какая-то техническая ошибка. 

In [8]:
df['dob_years'].value_counts()


35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

Нулевой возраст некорректен

**Вывод**

Выявили пропущенные значения в данных, определили некорректно заполненные столбцы, выявили неправильные типы данных для некоторых столбцов. 

## Шаг 2. Предобработка данных

### Обработка пропусков

In [9]:
df.children.value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

In [10]:
df['children'] = df['children'].replace([-1,20], [1, 2])

In [11]:
df['total_income'] = df['total_income'].fillna(0)

In [12]:
df['days_employed'] = df['days_employed'].fillna(0)

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [14]:
df.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

**Вывод**

Пропуски заменили на 0 (описано выше), количество детей привели в соответствии со здравым смыслом. Что делать с days_employed без пояснений со стороны дата инженера или разработчика непонятно, так как там явные ошибки.

### Замена типа данных

In [15]:
dob_years_mean = df['dob_years'].mean()
df['dob_years']= df['dob_years'].replace(0, dob_years_mean)
df['dob_years']= df['dob_years'].astype('int')
df['education'] = df['education'].str.lower()
df['total_income'] =  df['total_income'].astype('int')
df['days_employed'] =  df['days_employed'].astype('int')

**Вывод**

Образование в исходном файле было написано разным образом: использовались строчные и заглавные буквы. Перевели все к строчным.
Нулевой возраст заменили на средний, так как новорожденные не могут брать кредит. 
Возраст в данном случае целочисленный, нет дробных значений.
Зарплаты имеют также целочисленные значения, но они не были распознаны правильно из-за пропуска и Pandas определил по умолчанию поставил тип float64.
Общий трудовой стаж в днях также целочисленный тип.
Изменение типов данных применялось ко всему столбцу, поэтому применяли astype()

### Обработка дубликатов

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


In [17]:
df.duplicated().sum()

71

In [18]:
df = df.drop_duplicates()

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21454 entries, 0 to 21524
Data columns (total 12 columns):
children            21454 non-null int64
days_employed       21454 non-null int64
dob_years           21454 non-null int64
education           21454 non-null object
education_id        21454 non-null int64
family_status       21454 non-null object
family_status_id    21454 non-null int64
gender              21454 non-null object
income_type         21454 non-null object
debt                21454 non-null int64
total_income        21454 non-null int64
purpose             21454 non-null object
dtypes: int64(7), object(5)
memory usage: 2.1+ MB


**Вывод**

Дубликаты удалили с помощью удаления целых строк drop_duplicates(). Может дубликатов больше, но нет уникальных идентификаторов в таблице. 
Дубликатов не много и скорее всего их появление связано с человеческим фактором или какой-то технической ошибкой. 
У меня почему-то в этом месте регулярно выкидывало ошибку. Помогало только Kernel\Restart and Run all

### Лемматизация

In [20]:
from pymystem3 import Mystem
m = Mystem()
def lemm(text):
    lemmas = m.lemmatize(text)
    return lemmas 
df['purpose_lem'] = df['purpose'].apply(lemm)


In [21]:
uniq_lem = []
list_lem = list(df['purpose_lem'])
for row in list_lem:
    for i in row:
        if i not in uniq_lem:
            uniq_lem.append(i)
uniq_lem


['покупка',
 ' ',
 'жилье',
 '\n',
 'приобретение',
 'автомобиль',
 'дополнительный',
 'образование',
 'сыграть',
 'свадьба',
 'операция',
 'с',
 'на',
 'проведение',
 'для',
 'семья',
 'недвижимость',
 'коммерческий',
 'жилой',
 'строительство',
 'собственный',
 'подержать',
 'свой',
 'со',
 'заниматься',
 'сделка',
 'подержанный',
 'получение',
 'высокий',
 'профильный',
 'сдача',
 'ремонт']

**Вывод**

Произвели поиск основных ключевых слов с помощью леммитизации. Для этого добавили новый столбец с основами слов. После чего выявили уникальные леммы. Их оказалось не так много,  что позволило определить основные цели для кредитов.  

### Категоризация данных

In [22]:
def categorization(data):
    if 'сдача' in data:
        return 'Недвижеммость для сдачи'
    elif 'автомобиль' in data:
        return 'Покупка автомобиля'
    elif 'свадьба' in data:
        return 'Свадьба'
    elif 'образование' in data:
        return 'Образование'
    elif 'коммерческий' in data:
        if 'недвижимость' in data:
             return 'Операции с комерческой недвижемостью'
        elif 'операция' in data:
             return 'Операции с комерческой недвижемостью'
        elif 'ремонт' in data:
             return 'Ремонт комерческой недвижемости'
        else:
            return 'Прочая коммерческая деятельность'
    elif ('жилье' in data) or ('жилой' in data):
        if 'ремонт' in data:
            return 'Ремонт в квартитере'
        elif 'строительство' in data:
            return 'Строительство дома'
        elif 'покупка' in data:
            return 'Покупка жилья'
        else:
            return 'Прочие операции с жилой недвижимостью'
    elif 'недвижимость' in data:
        return 'Приобретение недвижимости'
    else:
        return -1
    

In [23]:
df['categorization'] = df['purpose_lem'].apply(categorization)

In [24]:
df.categorization.value_counts()

Покупка автомобиля                       4306
Образование                              4013
Приобретение недвижимости                3810
Покупка жилья                            2510
Свадьба                                  2324
Операции с комерческой недвижемостью     1311
Прочие операции с жилой недвижимостью    1298
Недвижеммость для сдачи                   651
Строительство дома                        624
Ремонт в квартитере                       607
Name: categorization, dtype: int64

In [25]:
def zp(data):
    if data == 0:
        return "Без подтвержденного дохода"
    elif data < 100000:
        return "Нижний сегмент доходов"
    elif 100000<= data < 135000:
        return "Средний сегмент доходов"
    elif 135000<= data < 170000:
        return "Высокий сегмент доходов"
    elif 170000<= data < 220000:
        return "Очень высокий сегмент доходов"
    elif data >= 220000:
        return "Премиум"
    else:
        return -1

In [26]:
df['total_income_categor'] = df['total_income'].apply(zp)
df['total_income_categor'].value_counts()

Нижний сегмент доходов           4463
Средний сегмент доходов          4075
Премиум                          4007
Высокий сегмент доходов          3597
Очень высокий сегмент доходов    3209
Без подтвержденного дохода       2103
Name: total_income_categor, dtype: int64

**Вывод**

Произвели категоризацию данных по цели кредита. В первоначальной таблице цели кредитов были написаны в свободной форме, для их корректного сравнения произвели категоризацию с помощью введения дополнительного столбца. Все возможные варианты описания кредитов свели к нескольким группам. Все строки категоризированы. 

Произвели категоризацию данных по доходам. Для корректного сравнения клиентов по доходам сократили все множество доходов до нескольких групп с целью удобства работы. Все строки категоризированы

Категоризацию производили таким образом, чтобы в каждой группе было по крайней мере несколько сотен объектов, чтобы можно было делать какие-то статистические выводы.


## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [27]:
df.groupby('children')['debt'].sum()/df.groupby('children')['debt'].count()

children
0    0.075438
1    0.091658
2    0.094925
3    0.081818
4    0.097561
5    0.000000
Name: debt, dtype: float64

In [28]:
df.groupby('children')['debt'].count()


children
0    14091
1     4855
2     2128
3      330
4       41
5        9
Name: debt, dtype: int64

**Вывод**

Достаточно обосновано выводы можно сделать для 0-3 детей. Для 4-5 мало данных. С увеличение количества детей частота невозврата кредита в срок растет и достигает максимума при двух детях, но потом убывает при 3-х детях. Для трех детей выборка существенно меньше, этим может быть вызвано уменьшения частоты невозврата либо же это может быть связано с дополнительной социальной поддержкой многодетных семей. 

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [29]:
(df.groupby('family_status')['debt'].sum()/df.groupby('family_status')['debt'].count()).sort_values().to_frame()

,debt
family_status,
вдовец / вдова,0.065693
в разводе,0.071130
женат / замужем,0.075452
гражданский брак,0.093471
Не женат / не замужем,0.097509


**Вывод**

Лучше всего платят "вдовец / вдова", что скорее всего связано с тем, что медианный возраст этой категории 58 лет и скорее всего в этом возрасте люди принимают меньше импульсивных решений. Неожиданно разведённые вышли на второе место. Среднее положение занимают люди в браке. Значительный рост невозврата в срок для гражданских браков. И наиболее рискованные клиенты "не женаты и не замужем".

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [30]:
(df.groupby('total_income_categor')['debt'].sum()/df.groupby('total_income_categor')['debt'].count()).sort_values().to_frame()

,debt
total_income_categor,
Премиум,0.070876
Нижний сегмент доходов,0.079319
Без подтвержденного дохода,0.080837
Очень высокий сегмент доходов,0.081957
Средний сегмент доходов,0.083681
Высокий сегмент доходов,0.091465


**Вывод**

Ожидаемо на первом месте клиенты с максимальным доходом. Неожиданно "Высокий сегмент доходов" стал наиболее ненадежным. Таким образом связь между доходом и надежностью весьма неоднозначная. Клиенты без подтвержденного дохода оказались достаточно надежными.

- Как разные цели кредита влияют на его возврат в срок?

In [31]:
(df.groupby('categorization')['debt'].sum()/df.groupby('categorization')['debt'].count()).sort_values().to_frame()

,debt
categorization,
Ремонт в квартитере,0.057661
Покупка жилья,0.066932
Прочие операции с жилой недвижимостью,0.072419
Приобретение недвижимости,0.075066
Операции с комерческой недвижемостью,0.075515
Строительство дома,0.076923
Недвижеммость для сдачи,0.079877
Свадьба,0.080034
Образование,0.092200


**Вывод**

Наиболее надежный кредит для ремонта. Достаточно надежны все операция с недвижемостью: покупка жилья, приобретение недвижимости...
Наиболее рискованные это образование и покупка автомобиля. 

## Шаг 4. Общий вывод

Наиболее ярко возврат кредита в срок зависит от цели кредита, семейного положения. Сложным образом зависит от дохода. Наименее рискованные клиенты без детей. 